In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [2]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [3]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler


In [4]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [6]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    plt.plot(y_test, label='Actual')
    plt.plot(lstm_predictions, label='Predicted')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig(f'Ethereum_actual_vs_predicted_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history.history['loss'], label='Training Loss')
    plt.plot(loss_history.history['val_loss'], label='Validation Loss')
    plt.title('Epoch Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'Ethereum_loss_curve_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.png')
    plt.close()
    
    return model, rmse


In [7]:
symbol = 'ETH-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [11]:
epochs = 100
batch_size = 4
sequence_length = 25
units = 50
layers = 2

In [12]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [13]:
print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
model = build_lstm_model(X_train, units=units, layers=layers)
model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2
Epoch 1/100
268/268 [==============================] - 23s 41ms/step - loss: 0.0071 - val_loss: 0.0043
Epoch 2/100
  4/268 [..............................] - ETA: 6s - loss: 0.0029

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 8s 29ms/step - loss: 0.0030 - val_loss: 3.9234e-04
Epoch 3/100
268/268 [==============================] - 8s 29ms/step - loss: 0.0021 - val_loss: 3.0869e-04
Epoch 4/100
268/268 [==============================] - 8s 28ms/step - loss: 0.0015 - val_loss: 5.2065e-04
Epoch 5/100
268/268 [==============================] - 8s 28ms/step - loss: 0.0012 - val_loss: 2.6246e-04
Epoch 6/100
268/268 [==============================] - 8s 29ms/step - loss: 0.0013 - val_loss: 5.9798e-04
Epoch 7/100
268/268 [==============================] - 8s 29ms/step - loss: 9.6432e-04 - val_loss: 1.7301e-04
Epoch 8/100
268/268 [==============================] - 8s 30ms/step - loss: 8.0683e-04 - val_loss: 1.6549e-04
Epoch 9/100
268/268 [==============================] - 8s 31ms/step - loss: 9.3030e-04 - val_loss: 1.3194e-04
Epoch 10/100
268/268 [==============================] - 8s 29ms/step - loss: 9.1233e-04 - val_loss: 1.6797e-04
Epoch 11/100
268/268 [===================

In [14]:
y_pred = model.predict(X_test)

9/9 [==============================] - 0s 19ms/step


In [18]:
y_pred = y_pred.reshape(1,-1)
y_test = y_test.reshape(1,-1)
y_pred_scaled = scaler.inverse_transform(y_pred)
y_test_scaled = scaler.inverse_transform(y_test)

In [19]:
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [20]:
calculate_mape(y_test_scaled, y_pred_scaled)

1.6174961642144035

In [21]:
np.sqrt(mean_squared_error(y_test_scaled, y_pred_scaled))

47.83943865651388

In [25]:
pd.DataFrame({'actual':y_test_scaled.flatten(), 'predicted':y_pred_scaled.flatten()}).head()

,actual,predicted
0,1870.789307,1836.940063
1,1904.651855,1869.908569
2,1877.704102,1904.427490
3,1995.060913,1881.620483
4,1900.221802,1985.485352


In [26]:
import joblib

In [27]:
joblib.dump(model, "Ethereum_model.pkl")

['Ethereum_model.pkl']